# Looping, part 6: Day 3 highs??

I'm going to try to look into whether the highs I've noticed a couple of times on the third day of her insulin pump have been happening consistently, or if it's just a total coincidence.

**UPDATE after a day or so of messing around:**

Wow, this was more difficult than I thought it would be. I just couldn't figure out where to find pod change times in the database. I ended up figuring out the EXACT time that her last pod change happened and then looked in the device status collection for that time stamp. (I just guessed that was the most likely collection where it might live). 

Turns out that when the new pod is paired for the first time, there's an new entry in the device status collection under "pump" with the rest of the pump information. It's called 'reservoir_display_override' and right after pairing it shows up with the value "Finish Pairing". 

There may be a better way to get at this but at least I didn't have to manually construct a list of pod change times. I'll take it for now!



In [2]:
import sys
sys.path.append("../")

from mdb_tools import load_data as ld
from mdb_tools import schemas
from mdb_tools import loop_stats as oop
from mdb_tools import sugar_plots as sp

import pandas as pd
import numpy as np

from datetime import datetime
import pytz

import matplotlib.pyplot as plt
%matplotlib widget

import hvplot.pandas


So this is just a little aside I guess, but as I work through these little exploration exercises, I keep running the same code over and over. So if I'm noticing I'm repeating the same things it seems like an obvious opportunity to write a function and same some space and some time. Not doing the function yet (bc I'm impatient to look at the day 3 highs!) but I am at least going to pull all the possible "function" stuff into one cell.

In [3]:
# Things that may change (ie eventual function arguments)
yml_secrets_file = '../../secrets/mdb_secrets.yml'
time_zone = 'US/Eastern'


# Access the database using the yml secrets file, and get a specific set of "collections"
col_entries, col_treatments, col_profile, col_devicestatus = ld.get_collections(yml_secrets_file)

# Grab schemas
entries_schema, treatments_schema, devicestatus_schema = schemas.mdb_schemas()

# Load
df_entries0 = col_entries.find_pandas_all({}, schema=entries_schema)
df_treatments = col_treatments.find_pandas_all({}, schema=treatments_schema)
df_devicestatus = col_devicestatus.find_pandas_all({}, schema=devicestatus_schema)

# Remove duplicate entries from cgm date - keep only loop for now.
df_entries = df_entries0[df_entries0["device"]=="loop://Dexcom/G6/21.0"].copy()

# Convert entries time strings to datetime format 
df_entries["time"] = pd.to_datetime(df_entries["dateString"]).dt.tz_convert(time_zone)
df_entries.set_index("time", inplace = True, drop=False)
df_entries.sort_index(inplace=True)

# Convert device status time strings to datetime format
df_devicestatus["time"] = pd.to_datetime(df_devicestatus["created_at"]).dt.tz_convert(time_zone)

In [63]:
pod_change_times = [x["time"] for xdex, x in df_devicestatus.iterrows() 
                    if x["pump"]['reservoir_display_override'] == 'Finish Pairing']

# Convert pod change times to unix
pod_times_unix = pd.Series(pod_change_times).view('int64').sort_values()

In [64]:
# Set up my start/end dates and start/end times
date_start = '2023-10-20'
date_end = '2023-11-06'

# start and end time in hours/time of day
time_start = 0
time_end = 24

# Extract only the dates of interest
df_sub = df_entries.loc[date_start:date_end].copy()

# make a column of just hour of day
df_sub["hour"] = df_sub.index.hour
df_sub["date"] = df_sub.index.date
df_sub["time_of_day"] = df_sub.index.time

# Set up color scale for date plotting
unq_dates = np.unique(df_sub.date)
date_dict = dict(zip(unq_dates, range(len(unq_dates))))
df_sub["date_color"] = df_sub["date"].map(date_dict)

# # Make another subset df that is just the requested hours
# df_req = df_sub[(df_sub["hour"] >= time_start) & (df_sub["hour"] <= time_end)]

# Convert requested input times to unix
in_times = df_sub.index.to_series()
in_times_unix = in_times.view('int64').sort_values()

In [65]:
# Get the indices of the most "recent" pod change times. Only keep the ones for the requested profile.
pod_idx = [(len([in_time - t for t_idx, t in enumerate(pod_times_unix) if
                 (in_time - t >= 0)]) - 1) for in_time in in_times_unix]

# Get the time since last pod change for each requested time
# pod_times_at_in_times = pod_change_times[pod_idx]
pod_times = [pod_change_times[p] for p in pod_idx]

df_sub["pod_times"] = pod_times

df_sub["pod_age_hours"] = (df_sub["time"] - df_sub["pod_times"]).dt.total_seconds()/3600


# Blood glucose vs pod age

This first plot is just the most basic - blood glucose ("sgv") vs pod age in hours. Not much to see here because all the points are overalapping. 

In [66]:
df_sub.hvplot.points(x="pod_age_hours", y="sgv")

:Points   [pod_age_hours,sgv]

We don't really want the extra data beyond 72 hours because those data aren't correct. It basically means that I missed a pod change indicator in the database. But I can assume that a pod change happened and just subtract 72 (or a multiple of 72 if need be) for pod ages beyond 72. It might not work all the time but it should work alright for us.

In [67]:
# A little modulo magic to just get the remainder 
df_sub["pod_age_hours_corr"] = [x % 72 for x in df_sub["pod_age_hours"]]

# And plot to see if it worked...
df_sub.hvplot.points(x="pod_age_hours_corr", y="sgv")

:Points   [pod_age_hours_corr,sgv]

In [74]:
# We only want just hours, and not decimal hours (for grouping later)
df_sub["pod_age_hours_floor"] = np.floor(df_sub["pod_age_hours_corr"])

df_sub.sort_values(by="pod_age_hours_floor", inplace=True)

## Point and line plot, colored by pod age

This one is mostly just to make sure I handled the pod age calculations correctly. It looks the way I expect - pod age increasing gradually and resetting every three days.

In [77]:
# Set up color scale for hours plotting
unq_items = np.unique(df_sub.pod_age_hours_floor)
item_dict = dict(zip(unq_items, range(len(unq_items))))
df_sub["hours_color"] = df_sub["pod_age_hours_floor"].map(item_dict)

df_sub.hvplot.points(x='time', y='sgv', color="hours_color", cmap="Blues", 
                     ylabel="Blood glucose (mg/dL)", xlabel="Time of day")


:Points   [time,sgv]   (hours_color)

## Boxplot with hourly bins

In this plot I'm grouping the data into hours to see if I can get a better view, using boxplot. I think that the main thing this plots shows is a twice-daily cycle that is probably more correlated with meals than with time since pod change. I think the focus is on the wrong scale in this case.


In [79]:
boxplot = df_sub.hvplot.box(y='sgv', by='pod_age_hours_floor', height=400, width=900, legend=False,
                           ylabel="Blood glucose (mg/dL)", xlabel="Pod age (hours)")
boxplot

:BoxWhisker   [pod_age_hours_floor]   (sgv)